In [ ]:
@variable(m, m_floor[d=9:11, t=9:21])  # mens floor
@variable(m, m_v[d=9:11, t=9:21]) # mens 
@variable(m, m_pb[d=9:11, t=9:21])
@variable(m, m_ph[d=9:11, t=9:21])
@variable(m, m_r[d=9:11, t=9:21])
@variable(m, m_hb[d=9:11, t=9:21])
@variable(m, w_f[d=9:11, t=9:21])
@variable(m, w_v[d=9:11, t=9:21])
@variable(m, w_ub[d=9:11, t=9:21])
@variable(m, w_b[d=9:11, t=9:21])
